In [7]:
import os
import uuid
import numpy as np
from flask import Flask, request, jsonify, render_template
import tensorflow as tf
from PIL import Image
from flask_cors import CORS
from jinja2 import TemplateNotFound 
import base64
# Path to the public folder
base_path = r'./temp/'

# Initialize Flask application
app = Flask(__name__, static_folder="public")
cors = CORS(app, origins=["http://localhost:3000"])

# Load the TensorFlow model
model_path = r"best_model.keras"
model = tf.keras.models.load_model(model_path)

@app.route('/process', methods=['POST'])
def upload_file():
    try:
        # Get the base64 encoded image data from the POST request
        data = request.form['file']
        # Decode the base64 image data
        image_data = base64.b64decode(data.split(',')[1])
        
        # Generate a UUID identifier
        c_uuid = str(uuid.uuid4())

        # Construct the file path
        filename = os.path.join(base_path, c_uuid + '.png')

        # Save the image data to a file
        with open(filename, 'wb') as f:
            f.write(image_data)

        # Load the image and convert to grayscale
        img = Image.open(filename).convert("L")

        # Resize the image
        new_image = img.resize((28, 28))

        # Convert image to array and resize
        x = np.array(new_image).reshape((28, 28, 1))
        x = np.expand_dims(x, axis=0)
        images = np.vstack([x])

        # Predict the digit in the image
        classes = model.predict(images, batch_size=1)

        # Get the predicted digit (class)
        result = int(np.argmax(classes))
        print("Output:", result)
        # Close the image
        img.close()

        # Remove the uploaded image
        os.remove(filename)

        # Return the result as JSON
        return jsonify({'value': result})

    except Exception as e:
        print("Error:", str(e))
        return jsonify({'error': str(e)}), 500

# Endpoint for digit recognition from uploaded image
@app.route('/', methods=['GET', 'POST'])
def start():
    try:
        return render_template(r'default.html')
    except TemplateNotFound:
        return "Template not found!", 404

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.106:5000
Press CTRL+C to quit
192.168.0.106 - - [04/Jul/2024 10:34:18] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step


192.168.0.106 - - [04/Jul/2024 10:34:19] "POST /process HTTP/1.1" 200 -


Output: 1
